<a href="https://colab.research.google.com/github/albeorla/HEIC-converter-notebook/blob/main/HEIC_to_JPEG_PNG_Converter_(Colab_%2B_Drive).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# @title HEIC to JPEG/PNG Converter for Google Colab (with Google Drive integration)
# @markdown This code will help you convert HEIC image files stored in your Google Drive to JPEG or PNG format.
# @markdown 1. Run the "Install Libraries & Mount Drive" cell. You'll be asked to authorize Colab to access your Google Drive.
# @markdown 2. Enter the path to the folder in your Google Drive containing the HEIC files in the "Set Input Folder Path" cell and run it.
# @markdown 3. Run the "Convert Files" cell. Choose your desired output format (JPEG or PNG).
# @markdown 4. Run the "Download Converted Files" cell. The converted images will be zipped and downloaded to your computer.

# --- Cell 1: Install Libraries & Mount Drive ---
# @title Install Libraries & Mount Drive
# @markdown Run this cell to install necessary libraries and connect to your Google Drive.

# Install necessary libraries (may take a minute)
print("Installing required libraries...")
# Need the development library for HEIF support
!sudo apt-get update && sudo apt-get install -y libheif-dev --quiet
# Install pyheif and Pillow
!pip install pyheif Pillow --quiet
print("Installation complete.")

# Import necessary libraries
import os
import glob
import zipfile
from google.colab import files, drive
from PIL import Image
import pyheif
import io

# Mount Google Drive
print("\nPlease authorize Colab to access your Google Drive.")
drive.mount('/content/drive')
print("\nGoogle Drive mounted successfully.")

# --- Cell 2: Set Input Folder Path ---
# @title Set Input Folder Path
# @markdown Enter the path to the folder in your Google Drive where your HEIC files are located.
# @markdown **How to get the path:**
# @markdown 1. Go to your Google Drive in your browser.
# @markdown 2. Navigate into the folder containing the HEIC files.
# @markdown 3. The path in the address bar might look like `https://drive.google.com/drive/folders/YOUR_FOLDER_ID`. You need the path *relative to your "My Drive"*.
# @markdown 4. Often, this path is simply the folder name if it's directly in "My Drive" (e.g., `MyFolder`), or nested if it's inside other folders (e.g., `Photos/Vacation2024`).
# @markdown 5. **Copy the folder name(s) exactly as they appear in Drive.**
# @markdown ---
# @markdown **Enter Google Drive Folder Path (relative to 'My Drive'):**
drive_folder_path = '' # @param {type:"string"}
# @markdown ---

# Construct the full path in the Colab environment
base_drive_path = '/content/drive/MyDrive/Photos'
source_dir = os.path.join(base_drive_path, drive_folder_path)

# Verify if the path exists
if os.path.isdir(source_dir):
    print(f"Input folder set to: {source_dir}")
    print("Please proceed to the next step (Convert Files).")
else:
    print(f"\nError: The specified path does not exist or is not a folder:")
    print(f"'{source_dir}'")
    print("\nPlease double-check the 'drive_folder_path' you entered.")
    print("Make sure it's the path *relative* to your 'My Drive' (e.g., 'FolderName' or 'Folder1/SubFolder'). Do not include '/content/drive/MyDrive/'.")


# --- Cell 3: Convert Files ---
# @title Convert Files
# @markdown Select the desired output format and run this cell to convert the files from the specified Google Drive folder.

# @markdown ---
# @markdown **Output Format:**
output_format = "PNG" # @param ["JPEG", "PNG"]
# @markdown ---

# Check if source_dir was set correctly in the previous cell
if 'source_dir' not in locals() or not os.path.isdir(source_dir):
    print("Error: Input folder path is not set or invalid. Please run the 'Set Input Folder Path' cell correctly first.")
else:
    convert_dir = '/content/converted_images' # Store converted images temporarily in Colab environment
    if not os.path.exists(convert_dir):
        os.makedirs(convert_dir)

    # Clear previous conversions if any
    for f in glob.glob(os.path.join(convert_dir, '*')):
        os.remove(f)

    print(f"\nLooking for HEIC files in: {source_dir}")
    print(f"Starting conversion to {output_format}...")
    conversion_count = 0
    conversion_errors = 0
    error_files = []

    target_dir = convert_dir

    # Get list of HEIC files from the specified Google Drive directory
    try:
        all_files = os.listdir(source_dir)
        heic_files = [f for f in all_files if f.lower().endswith('.heic')]
    except FileNotFoundError:
        print(f"Error: Could not access the directory '{source_dir}'. Please ensure the path is correct and you have permissions.")
        heic_files = [] # Prevent further errors
    except Exception as e:
        print(f"An unexpected error occurred while listing files: {e}")
        heic_files = [] # Prevent further errors


    if not heic_files:
        print("\nNo .HEIC files found in the specified directory.")
        if 'all_files' in locals():
             print(f"(Found {len(all_files)} total files/folders in the directory)")
    else:
        print(f"Found {len(heic_files)} HEIC files to convert.")
        for filename in heic_files:
            source_path = os.path.join(source_dir, filename)
            # Create the output filename
            base_name = os.path.splitext(filename)[0]
            if output_format == "JPEG":
                output_filename = f"{base_name}.jpg"
            else: # PNG
                output_filename = f"{base_name}.png"
            target_path = os.path.join(target_dir, output_filename)

            try:
                # Check if it's actually a file before trying to read
                if not os.path.isfile(source_path):
                    print(f"  Skipping '{filename}' as it is not a file (might be a folder).")
                    continue

                print(f"Converting {filename} to {output_filename}...")
                # Read the HEIC file
                heif_file = pyheif.read(source_path)

                # Convert to Pillow Image object
                image = Image.frombytes(
                    heif_file.mode,
                    heif_file.size,
                    heif_file.data,
                    "raw",
                    heif_file.mode,
                    heif_file.stride,
                )

                # Save the image in the desired format
                if output_format == "JPEG":
                    # Save as JPEG, handling potential transparency issues (convert to RGB)
                    if image.mode in ("RGBA", "P", "LA"): # Added LA mode check
                       image = image.convert("RGB")
                    image.save(target_path, "JPEG", quality=95) # Adjust quality as needed
                else: # PNG
                    image.save(target_path, "PNG")

                conversion_count += 1
            except pyheif.error.HeifError as he:
                 print(f"  Error converting {filename}: {he} - This might not be a valid HEIC file or could be corrupted.")
                 conversion_errors += 1
                 error_files.append(filename)
            except Exception as e:
                print(f"  Error converting {filename}: {e}")
                conversion_errors += 1
                error_files.append(filename)

        print(f"\nConversion complete.")
        print(f"Successfully converted: {conversion_count} files.")
        if conversion_errors > 0:
            print(f"Failed to convert: {conversion_errors} files:")
            for fn in error_files:
                print(f"  - {fn}")

# --- Cell 4: Download Converted Files ---
# @title Download Converted Files
# @markdown Run this cell to download the converted images as a ZIP file.

# Check if convert_dir exists and has been populated
if 'convert_dir' not in locals() or not os.path.exists(convert_dir):
     print("Converted images directory not found. Please run the conversion step first.")
else:
    zip_filename = 'converted_images.zip'
    converted_files_list = os.listdir(convert_dir)

    if not converted_files_list:
        print("No converted files found to download. Check the conversion step for errors.")
    else:
        print(f"\nZipping {len(converted_files_list)} converted files...")
        # Use /content/ as the base path for the zip file to avoid permission issues
        zip_path = f'/content/{zip_filename}'
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for file_to_zip in converted_files_list:
                file_path = os.path.join(convert_dir, file_to_zip)
                zipf.write(file_path, arcname=file_to_zip) # arcname avoids including the folder structure

        print(f"Zip file '{zip_filename}' created. Starting download...")
        files.download(zip_path)
        print("Download initiated.")

Installing required libraries...
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists...
Building dep

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download initiated.
